In [1]:
import warnings
warnings.filterwarnings("ignore")  # Ignore all warnings
from typing import List, Dict, Any

import numpy as np
# import matplotlib.pyplot as plt
from agentic_energy.language_models.basic_llm_amap import records_to_arrays, llm_solve, llm_solve_from_records, solve_daily_llm
from agentic_energy.data_loader import EnergyDataLoader, BatteryDataLoader
from agentic_energy.schemas import BatteryParams, DayInputs, EnergyDataRecord, SolveRequest, SolveFromRecordsRequest

Forecast Engine using device: cpu


#### Forecast Data Load

In [2]:
italy_forecast_obj = EnergyDataLoader(region = "ITALY", data_version="forecast", forecast_type="RF")
italy_forecast_data = italy_forecast_obj.load_region_data()
italy_forecast = await italy_forecast_obj.get_filtered_data("2019-01-01", "2019-01-01")

2025-10-16 15:39:33.053 | DEBUG    | agentics.core.agentics:from_csv:307 - Importing Agentics of type EnergyDataRecord from CSV c:\Users\16467\OneDrive\Desktop\Columbia\Agentics\Another\Agentics_for_EnergyArbitrage_Battery\energy_arbitrage\agentic_energy\data\Italy_data_forecast_RF.csv


In [3]:
stats_forecast_consumption = await EnergyDataLoader.get_summary_stats_from_ag(italy_forecast_data, column="consumption")
print(stats_forecast_consumption.pretty_print())

aType : <class 'agentic_energy.schemas.MetricStats'>
count: 24
min: 21.64553853
max: 36.33831261
avg: 27.800151357916672
median: 28.037061115
p25: 24.763189914999998
p75: 29.8596689575
std: 4.128303881165521
var: 17.0428929352463


aType : <class 'agentic_energy.schemas.MetricStats'>
count: 24
min: 21.64553853
max: 36.33831261
avg: 27.800151357916672
median: 28.037061115
p25: 24.763189914999998
p75: 29.8596689575
std: 4.128303881165521
var: 17.0428929352463




##### Actual Data Load

In [4]:
italy_actual_obj = EnergyDataLoader(region = "ITALY", data_version="actual")
italy_actual_data = italy_actual_obj.load_region_data()
italy_actual = await italy_actual_obj.get_filtered_data("2019-01-01", "2019-01-01")

2025-10-16 15:39:33.535 | DEBUG    | agentics.core.agentics:from_csv:307 - Importing Agentics of type EnergyDataRecord from CSV c:\Users\16467\OneDrive\Desktop\Columbia\Agentics\Another\Agentics_for_EnergyArbitrage_Battery\energy_arbitrage\agentic_energy\data\Italy_data_actual.csv


In [5]:
stats_actual_consumption = await EnergyDataLoader.get_summary_stats_from_ag(italy_actual_data, column="consumption")
print(stats_actual_consumption.pretty_print())

aType : <class 'agentic_energy.schemas.MetricStats'>
count: 24
min: 19.3
max: 30.6
avg: 24.475416666666664
median: 24.490000000000002
p25: 22.0325
p75: 25.97
std: 3.2926040595637303
var: 10.841241493055556


aType : <class 'agentic_energy.schemas.MetricStats'>
count: 24
min: 19.3
max: 30.6
avg: 24.475416666666664
median: 24.490000000000002
p25: 22.0325
p75: 25.97
std: 3.2926040595637303
var: 10.841241493055556




In [6]:
stats_forecast_consumption.states[0].p25

24.763189914999998

#### Battery Config Load

In [7]:
batt = BatteryDataLoader(
    load_stats={
        "p25":stats_forecast_consumption.states[0].p25,
        "p75":stats_forecast_consumption.states[0].p75
    },
    duration_hours=4,
    soc_init=0.5,
    soc_min=0.0,
    soc_max=1.0,
    eta_c = 0.95,
    eta_d = 0.95,
    soc_target=0.5,
)
battery = batt.compute_battery_params()
print(batt.summary())

{'Capacity (kWh)': 20.39, 'Charge Power (kW)': 5.1, 'Discharge Power (kW)': 5.1, 'Efficiency (Charge/Discharge)': (0.95, 0.95), 'Duration (hours)': 4}


In [8]:
result_llm = []
day_info = []
decision_list = []
soc_list = []  # Initialize soc_list

for i in range(len(italy_forecast_data)//24):
    print(f"Solving day {i+1} of {len(italy_forecast_data)//24}")

    forecast_prices, forecast_demand = records_to_arrays(italy_forecast_data[i*24:(i+1)*24])
    actual_prices, actual_demand = records_to_arrays(italy_actual_data[i*24:(i+1)*24])
    day = DayInputs(
        prices_buy=actual_prices,
        demand_kw=actual_demand,
        prices_sell=actual_prices,
        allow_export=True,
        dt_hours=1.0,
        prices_buy_forecast=forecast_prices,
        demand_kw_forecast=forecast_demand,
        prices_sell_forecast=forecast_prices,
    )
    day_info.append(day)
    req = SolveRequest(
        battery=battery,
        day=day,
        solver=None,
        solver_opts=None
    )
    res = await solve_daily_llm(req)
    result_llm.append(res)
    decision_list += res.decision 
    battery.soc_init = res.soc[-1]
    soc_list += res.soc[:-1]

Solving day 1 of 1




╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 8d7cb26a-36ed-4340-9709-ea19f8643258                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Task Executor                                                                                           │
│                                                                                                                 │
│  Task:                                                                                                          │
│  Your task is to transduce a source Pydantic Object into the specified Output type. Generate only slots that    │
│  are logically deduced from the input information, otherwise live then null.                                    │
│                                                                                                                 │
│  Read carefully the following instructions for executing your task:                                             │
│                                                                                                                 │
│          You are solving a daily battery scheduling optimization problem using forecast-based reasoning and     │
│  constraint satisfaction.                                                                                       │
│                                                                                                                 │
│          You are provided with both forecasted and actual market data:                                          │
│                                                                                                                 │
│          FORECAST INPUTS (for decision-making):                                                                 │
│              - Forecasted buying prices: [ 46.56980749  39.53773847  37.0457292   33.97309305  33.77742699      │
│    38.45316148  46.85447987  54.84395765  58.57893156  53.37109246                                              │
│    53.19388942  50.15505505  49.06797744  50.47452     52.19668328                                              │
│    59.96764545  66.90326064  61.94714286  65.59960145 105.25                                                    │
│    68.6268938   63.09534354  58.80213862  47.13727199]  (array of length T)                                     │
│              - Forecasted selling prices: [ 46.56980749  39.53773847  37.0457292   33.97309305  33.77742699     │
│    38.45316148  46.85447987  54.84395765  58.57893156  53.37109246                                              │
│    53.19388942  50.15505505  49.06797744  50.47452     52.19668328                                              │
│    59.96764545  66.90326064  61.94714286  65.59960145 105.25                                                    │
│    68.6268938   63.09534354  58.80213862  47.13727199]  (array of length T)                                     │
│              - Forecasted demand: [25.26011461 24.04358788 22.65896303 21.64553853 22.08336775 22.12202695      │
│   22.14666667 25.12610667 28.17412223 29.68239749 27.41691718 29.55699994                                       │
│   29.56387416 27.9        27.81625    28.89797657 30.39148336 31.63044248                                       │
│   35.75480032 36.33831261 32.71125    31.91367603 29.36570087 25.00305726]  (array of length T)                 │
│                                                                                                                 │
│          ACTUAL INPUTS (for ex-post evaluation):                                                                │
│              - Realized buying prices: [51.   46.27 39.78 27.87 22.   20.   20.9  26.54 25.   25.   34.64       │
│  41.03                                                                                                          │
│   48.97 41.06 37.95 53.59 58.65 64.64 65.   66.26 65.99

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Task Executor                                                                                           │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  ```json                                                                                                        │
│  {                                                                                                              │
│    "status": "success",                                                                                         │
│    "message": "The battery charges when forecast prices are below average and discharges when above,            │
│  respecting SoC and power limits. Demand is met at all times while minimizing realized cost. The final SoC is   │
│  maintained near the target.",                                                                                  │
│    "objective_cost": 7962.5766,                                                                                 │
│    "charge_kw": [                                                                                               │
│      0.0,                                                                                                       │
│      0.0,                                                                                                       │
│      0.0,                                                                                                       │
│      5.1,                                                                                                       │
│      5.1,                                                                                                       │
│      0.0,                                                                                                       │
│      0.0,                                                                                                       │
│      0.0,                                                                                                       │
│      0.0,                                                                                                       │
│      0.0,                                                                                                       │
│      0.0,                                                                                                       │
│      0.0,                                                                                                       │
│      0.0,                                                                                                       │
│      0.0,                                                                                                       │
│      0.0,                                                                                                       │
│      0.0,                                                                                                       │
│      0.0,                                                                                                       │
│      0.0,                                                                                                       │
│      0.0,                                                                                                       │
│      0.0,                                                                                                       │
│      0.0,                                                                                                       │
│      0.0,                                              

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 68c4e5d1-0106-4d90-ab94-b6cb8358a45f                                                                     │
│  Agent: Task Executor                                                                                           │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 8d7cb26a-36ed-4340-9709-ea19f8643258                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: ```json                                                                                          │
│  {                                                                                                              │
│    "status": "success",                                                                                         │
│    "message": "The battery charges when forecast prices are below average and discharges when above,            │
│  respecting SoC and power limits. Demand is met at all times while minimizing realized cost. The final SoC is   │
│  maintained near the target.",                                                                                  │
│    "objective_cost": 7962.5766,                                                                                 │
│    "charge_kw": [                                                                                               │
│      0.0,                                                                                                       │
│      0.0,                                                                                                       │
│      0.0,                                                                                                       │
│      5.1,                                                                                                       │
│      5.1,                                                                                                       │
│      0.0,                                                                                                       │
│      0.0,                                                                                                       │
│      0.0,                                                                                                       │
│      0.0,                                                                                                       │
│      0.0,                                                                                                       │
│      0.0,                                                                                                       │
│      0.0,                                                                                                       │
│      0.0,                                                                                                       │
│      0.0,                                                                                                       │
│      0.0,                                                                                                       │
│      0.0,                                                                                                       │
│      0.0,                                                                                                       │
│      0.0,                                                                                                       │
│      0.0,                                                                                                       │
│      0.0,                                                                                                       │
│      0.0,                                             


✓ Optimization successful
aType : <class 'agentic_energy.schemas.SolveResponse'>
status: success
message: The battery charges when forecast prices are below average and discharges
  when above, respecting SoC and power limits. Demand is met at all times while minimizing
  realized cost. The final SoC is maintained near the target.
objective_cost: 7962.5766
charge_kw:
- 0.0
- 0.0
- 0.0
- 5.1
- 5.1
- 0.0
- 0.0
- 0.0
- 0.0
- 0.0
- 0.0
- 0.0
- 0.0
- 0.0
- 0.0
- 0.0
- 0.0
- 0.0
- 0.0
- 0.0
- 0.0
- 0.0
- 0.0
- 0.0
discharge_kw:
- 0.0
- 0.0
- 0.0
- 0.0
- 0.0
- 0.0
- 0.0
- 0.0
- 0.0
- 0.0
- 0.0
- 0.0
- 0.0
- 0.0
- 0.0
- 0.0
- 0.0
- 5.1
- 5.1
- 5.1
- 5.1
- 5.1
- 5.1
- 0.0
import_kw:
- 22.74
- 21.5
- 20.5
- 14.54
- 14.2
- 19.65
- 20.86
- 22.21
- 23.13
- 24.38
- 25.29
- 25.88
- 26.09
- 24.73
- 24.27
- 24.6
- 25.9
- 34.21
- 35.21
- 35.7
- 34.56
- 33.17
- 31.03
- 23.46
export_kw:
- 0.0
- 0.0
- 0.0
- 0.0
- 0.0
- 0.0
- 0.0
- 0.0
- 0.0
- 0.0
- 0.0
- 0.0
- 0.0
- 0.0
- 0.0
- 0.0
- 0.0
- 0.0
- 0.0
- 0.0

In [9]:
# import numpy as np
# import matplotlib.pyplot as plt

# hours = np.arange(24)
# fig, ax1 = plt.subplots(figsize=(14, 6))

# # --- Primary axis (Prices) ---
# ax1.plot(hours, day_info[0].prices_buy, '--b', label='Prices')
# ax1.set_xlabel('Hour')
# ax1.set_ylabel('Prices ($/MWh)', color='b')
# ax1.tick_params(axis='y', labelcolor='b')

# # --- Secondary axis (State of Charge) ---
# ax2 = ax1.twinx()
# ax2.plot(hours, np.array(result_llm[0].soc[1:]) * battery.capacity_kwh,
#          '-og', label='State of Charge (kWh)', linewidth=2, markersize=6)
# ax2.set_ylabel('State of Charge (kWh)', color='g')
# ax2.tick_params(axis='y', labelcolor='g')


# # --- Legends ---
# lines_1, labels_1 = ax1.get_legend_handles_labels()
# lines_2, labels_2 = ax2.get_legend_handles_labels()
# ax2.legend(lines_1 + lines_2, labels_1 + labels_2, loc='best', frameon=False)

# plt.title('Prices vs State of Charge (SoC) Over Time')
# plt.grid(alpha=0.3)
# plt.tight_layout()
# plt.show()


In [10]:
# import numpy as np
# import matplotlib.pyplot as plt

# hours = np.arange(24)

# # Data arrays
# load = np.array(day_info[0].demand_kw)
# charge = np.array(result_llm[0].charge_kw)
# discharge = np.array(result_llm[0].discharge_kw)
# import_kw = np.array(result_llm[0].import_kw)
# export_kw = np.array(result_llm[0].export_kw) if getattr(day_info[0], "allow_export", False) else np.zeros_like(load)

# # Flip battery convention for visualization
# charge_vis = -charge       # show charge below 0
# discharge_vis = discharge  # show discharge above 0

# # ---------------------- PLOT ----------------------
# fig, ax = plt.subplots(figsize=(14, 6))

# # --- positive side stacking (Import + Discharge + Export) ---
# # import is base layer
# ax.bar(hours, import_kw, width=0.5, color='orange', label='Import (+)')
# # discharge stacked on top of import
# ax.bar(hours, discharge_vis, width=0.5, bottom=import_kw, color='indianred', label='Discharge (+)')
# # export stacked on top (for clear separation)
# ax.bar(hours, export_kw, width=0.5, bottom=import_kw + discharge_vis, color='limegreen', alpha=0.7, label='Export (+)')

# # --- negative side stacking (Charge only) ---
# ax.bar(hours, charge_vis, width=0.5, color='cornflowerblue', label='Charge (−)')

# # --- Load as line ---
# ax.plot(hours, load, '-ok', linewidth=2, markersize=5, label='Load (kW)')

# # Formatting
# ax.axhline(0, color='black', linewidth=0.8)
# ax.set_ylabel("Power (kW)")
# ax.set_xlabel("Hour")
# ax.set_title("Power Balance Components (Charge ↓  |  Discharge ↑)")
# ax.legend(ncol=3, loc='upper left', fontsize=9)
# ax.grid(alpha=0.3)

# plt.tight_layout()
# plt.show()


In [11]:
total_objective_value = 0
for day_result in result_llm:
    total_objective_value += day_result.objective_cost
print(f"Total Objective Value over all days: ${total_objective_value:.2f} for {len(result_llm)} days")

Total Objective Value over all days: $7962.58 for 1 days


# Checking LLM Connection

In [1]:
# Reload environment variables and test with GPT-3.5-turbo
import os
from dotenv import load_dotenv
load_dotenv()  # Reload environment variables from .env file

# Test the LLM connection
from agentics.core.llm_connections import get_llm_provider
try:
    llm = get_llm_provider("ollama")
    response = llm.call("Hello! Can you briefly introduce yourself? (including what LLM model you are: gpt4, 4o, mini, or what exact model)")
    print("✅ LLM Connection Successful!")
    print(f"Response: {response}")
except Exception as e:
    print(f"❌ LLM Connection Failed: {e}")
    print("\nTroubleshooting options:")
    print("1. Check if your OpenAI API key is valid")
    print("2. Ensure you have sufficient OpenAI credits")
    print("3. Try using gpt-3.5-turbo instead of gpt-4")
    print("4. Consider using Gemini as an alternative")

2025-11-18 00:38:37.476 | DEBUG    | agentics.core.llm_connections:get_llm_provider:59 - Using specified LLM provider: ollama


✅ LLM Connection Successful!
Response: I am a large language model created by Anthropic to be helpful, harmless, and honest.
